In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-nlp") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
    .getOrCreate()

In [5]:
# Check the SparkSession
spark

In [177]:
sc = spark.sparkContext
data = sc.wholeTextFiles("s3a://zihe-public/articles/AA/wiki_00")

In [179]:
import re
def get_id(x):
    pattern = "\<doc\sid\=\"(\d+)\"(.*)title\=\"(.*)\"\>"
    pattern_re = re.compile(pattern)
    matches = pattern_re.search(x)
    if not matches:
        return ""
    return matches[1]

def get_title(x):
    pattern = "\<doc\sid\=\"(\d+)\"(.*)title\=\"(.*)\"\>"
    pattern_re = re.compile(pattern)
    matches = pattern_re.search(x)
    if not matches:
        return ""
    return matches[3]

def get_content(x):
    pattern = "\>\\n(.*?)\\n{2}(.*)"
    pattern_re = re.compile(pattern)
    matches = pattern_re.search(x)
    if not matches:
        return ""
    return matches[2]

In [180]:
pages = data.flatMap(lambda x: (x[1].split('</doc>'))).map(lambda x : (get_id(x), get_title(x), get_content(x)))

In [182]:
pages = pages.toDF(["id","title","text"])

In [184]:
pages.show(2)

+---+---------+--------------------+
| id|    title|                text|
+---+---------+--------------------+
| 12|Anarchism|Anarchism is a po...|
| 25|   Autism|Autism is a devel...|
+---+---------+--------------------+
only showing top 2 rows

